In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
from datetime import date

In [4]:
from dateutil.relativedelta import relativedelta

In [5]:
from bokeh.plotting import figure, show, output_notebook

In [6]:
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, \
    Range1d, HoverTool, CrosshairTool

In [7]:
apikey = "E456SMCHeyee8d_sF4Sv"

In [8]:
urlhead = "https://www.quandl.com/api/v3/datatables/WIKI/PRICES.json?ticker="
urldate = "&date.gte="
urlcols = "&qopts.columns="
urltail = "&api_key=" + apikey

In [9]:
monthago = date.today() + relativedelta(months=-1)
date = monthago.strftime("%Y%m%d")

In [10]:
cols = "date,close"

In [12]:
def plot_ticker(ticker):
    # Retrieve and process data
    url = urlhead + ticker + urldate + date + urlcols + cols + urltail
    page = requests.get(url)
    json = page.json()
    df = pd.DataFrame(json['datatable']['data'], columns=['date','close'])
    df['date'] = pd.to_datetime(df['date'])
    df['date_str'] = df['date'].map(lambda x: x.strftime("%Y-%m-%d"))
    dfcds = ColumnDataSource(df)
    
    # Create Bokeh plot
    p = figure(width=600, height=300, title=ticker.upper(), tools="")

    hover = HoverTool(tooltips = [
        ('Date', '@date_str'),
        ('Close', '@close')
    ])
    hover.mode = 'vline'
    hover.line_policy = 'nearest'
    p.add_tools(hover)

    crosshair = CrosshairTool()
    crosshair.dimensions = 'height'
    p.add_tools(crosshair)

    p.line('date', 'close', source = dfcds)

    p.xaxis.formatter=DatetimeTickFormatter(days=["%d %b"])
    p.x_range=Range1d(df['date'].min(), df['date'].max())

    p.toolbar.logo = None
    p.toolbar_location = None

    return p

In [13]:
output_notebook()
fig = plot_ticker('goog')
show(fig)

Loading BokehJS ...

/home/mike/Documents/pyvenv/pydata2/lib/python2.7/site-packages/bokeh/core/json_encoder.py:73: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  if pd and isinstance(obj, pd.tslib.Timestamp):
